0=women
1=men
0=no pd
1=pd

In [1]:
## Initial Imports

%matplotlib inline

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import LeaveOneOut, GridSearchCV, KFold, train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE

**`Read data and Display data dimension`**

In [44]:
pd_speech_features = pd.read_csv('pd_speech_features.csv')
new_header = pd_speech_features.iloc[0] #grab the first row for the header
pd_speech_features = pd_speech_features[1:] #take the data less the header row
pd_speech_features.columns = new_header #set the header row as the df header
pd_speech_features.head()
print('The shape of the matrix is :', pd_speech_features.shape)

The shape of the matrix is : (756, 755)


#### Gather most general metadata about the data

In [45]:
pd_speech_features['patient/healthy count'] = 1
pd_speech_features.groupby('class').sum()/3

,patient/healthy count
class,
0,64.0
1,188.0


In [46]:
#pd_speech_features['patient/healthy count'] = 1
pd_speech_features['gender'].value_counts()

1    390
0    366
Name: gender, dtype: int64

In [47]:
pd_speech_features[['gender','class']]=pd_speech_features[['gender','class']].astype('int')
pd_speech_features.groupby(by='gender')['class'].sum()/3

gender
0     81.0
1    107.0
Name: class, dtype: float64

In [48]:
pd_speech_features = pd_speech_features.drop(['patient/healthy count'], axis = 1)  #756x755

All the data was imported in the object type, we now need to take care about the types in the dataframe

In [49]:
pd_speech_features =  pd_speech_features.astype(float) #per default all floats 
pd_speech_features[['id', 'numPulses', 'numPeriodsPulses']] = pd_speech_features[['id', 'numPulses', 'numPeriodsPulses']].astype(int) #ints
pd_speech_features[['gender', 'class']] = pd_speech_features[['gender', 'class']].astype('category') #categoricals
pd_speech_features.dtypes

0
id                              int32
gender                       category
PPE                           float64
DFA                           float64
RPDE                          float64
                               ...   
tqwt_kurtosisValue_dec_33     float64
tqwt_kurtosisValue_dec_34     float64
tqwt_kurtosisValue_dec_35     float64
tqwt_kurtosisValue_dec_36     float64
class                        category
Length: 755, dtype: object

In [50]:
df=pd_speech_features

In [51]:
class Htest:
    def __init__(self, population_mean_diff=0, p=0.05, cls_names='class', test_type='z_test'):
        
        '''
            cls_vals=target class values
            p=confidence interval, default=0.05
            
            population_mean_diff: 
                        H0(null hypothesis): difference of population mean over feature F, default=0
            
            cls_names=sample classes
            
            test_type: what kind of hypothesis testing
                            
        '''
        
        self.p=p
        self.cls=cls_names
        self.population_mean_diff=population_mean_diff
        
    def z_test(self, df1, df2, threshold_val=1.96, show_top=1):
        '''
        Assumptions:
                    1. sample size=population size
                    2. normal distribution
        input:
                df1=dataframe of numerical features of sample1
                df2=dataframe of numerical features of sample2
                threshold_val= z score of 1.96 for 95% confidence(p<0.05)
        '''
        self.df1=df1
        self.df2=df2
        
        std_error1=(self.df1.std().pow(2.))/self.df1.shape[0]
        std_error2=(self.df2.std().pow(2.))/self.df2.shape[0]
        
        denominator=(std_error1+std_error2).pow(1./2)
        
        nominator=(self.df1.mean()-self.df2.mean())\
                    -self.population_mean_diff
        
        z_scores=nominator/denominator
        feature_z=z_scores[z_scores.abs()>threshold_val].abs()
        
        return feature_z
        

In [52]:
htest=Htest()

In [53]:
dfk=df.kurtosis().between(-1,1) 
dfs=df.skew().between(-1,1, inclusive=True)

normals=set(dfk.index)-(set(dfk.index)-set(dfs.index))
print(len(normals))

753


C:\Users\User\AppData\Local\Temp/ipykernel_10896/360618407.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dfk=df.kurtosis().between(-1,1)
C:\Users\User\AppData\Local\Temp/ipykernel_10896/360618407.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dfs=df.skew().between(-1,1, inclusive=True)
C:\Users\User\AppData\Local\Temp/ipykernel_10896/360618407.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  dfs=df.skew().between(-1,1, inclusive=True)


In [54]:
def sk(df):
    dfk=df.kurtosis().between(-1,1) 
    dfs=df.skew().between(-1,1, inclusive=True)
    
    ktrue=[]
    strue=[]
    
    for i,j in zip(dfs.index,dfs.values): 
        if j==True:
            #print(i,' ',j)
            strue.append(i)
    
    print('skewness:',len(strue))
    
    for i,j in zip(dfk.index,dfk.values): 
        if j==True:
            #print(i,' ',j)
            ktrue.append(i)
    
    print('kurtosis:',len(ktrue))
    
    return set(strue),set(ktrue)

In [55]:
sn,kn=sk(df)
fns=sn.intersection(kn)
print(len(fns))

skewness: 227
kurtosis: 151
147


C:\Users\User\AppData\Local\Temp/ipykernel_10896/3755790733.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dfk=df.kurtosis().between(-1,1)
C:\Users\User\AppData\Local\Temp/ipykernel_10896/3755790733.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dfs=df.skew().between(-1,1, inclusive=True)
C:\Users\User\AppData\Local\Temp/ipykernel_10896/3755790733.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  dfs=df.skew().between(-1,1, inclusive=True)


In [56]:
#sn.sort_values(ascending=False).head(50)

In [57]:
print('id' in list(fns))
print('gender' in list(fns))
print('class' in list(fns))

False
False
False


In [58]:
print(fns)
Fh=fns

{'tqwt_kurtosisValue_dec_36', 'det_entropy_log_10_coef', 'tqwt_entropy_log_dec_28', 'tqwt_meanValue_dec_26', 'tqwt_entropy_shannon_dec_21', 'mean_MFCC_1st_coef', 'tqwt_meanValue_dec_18', 'tqwt_meanValue_dec_30', 'tqwt_entropy_log_dec_35', 'mean_MFCC_11th_coef', 'det_entropy_log_9_coef', 'mean_8th_delta_delta', 'app_LT_TKEO_std_2_coef', 'app_LT_entropy_shannon_3_coef', 'mean_12th_delta_delta', 'tqwt_minValue_dec_18', 'tqwt_skewnessValue_dec_36', 'tqwt_meanValue_dec_24', 'std_12th_delta', 'app_LT_TKEO_std_9_coef', 'tqwt_maxValue_dec_14', 'f2', 'VFER_NSR_SEO', 'std_9th_delta_delta', 'app_LT_TKEO_mean_5_coef', 'std_10th_delta', 'tqwt_entropy_log_dec_11', 'tqwt_meanValue_dec_17', 'tqwt_meanValue_dec_23', 'app_LT_TKEO_mean_6_coef', 'app_LT_TKEO_mean_7_coef', 'tqwt_entropy_log_dec_14', 'tqwt_meanValue_dec_20', 'tqwt_entropy_log_dec_33', 'tqwt_meanValue_dec_16', 'std_10th_delta_delta', 'std_8th_delta_delta', 'tqwt_maxValue_dec_15', 'tqwt_entropy_log_dec_10', 'app_LT_entropy_shannon_9_coef', 't

In [59]:
fns=list(fns)

if 'gender' not in fns:
    fns.append('gender')
    
if 'class' not in fns:
    fns.append('class')

# adding the class feature to 147 selected features
Fh=list(Fh)

if 'class' not in Fh:
    Fh.append('class')

dfn=df.loc[:,Fh]
print(dfn.shape)
#dfn=dfn[dfn['class']==1]
#print(dfn.shape)

(756, 148)


In [60]:
len(fns)

149

In [61]:
# people who already have Pd
#df1=dfn[(dfn['gender']==0) & (dfn['class']==1)].drop(['gender','class'],axis=1)
#df2=dfn[(dfn['gender']==1) & (dfn['class']==1)].drop(['gender','class'],axis=1)
df1=dfn[(dfn['class']==1)].drop(['class'],axis=1)
df2=dfn[(dfn['class']==0)].drop(['class'],axis=1)

In [62]:
#fg=htest.z_test(df1,df2) # selected features hy hypothesis test hg
fh=htest.z_test(df1,df2)

In [63]:
fh.shape

(95,)

In [64]:
print('class' in fh)

False


In [65]:
fh_list=list(fh.index)

In [66]:
print('class' in fh_list)

False


In [67]:
if 'class' not in fh_list:
    fh_list.append('class')

print('class' in fh_list)

True


In [68]:
print(df[fh_list].shape)


(756, 96)


In [69]:
FPD=df[fh_list]
FPD.head()

,tqwt_kurtosisValue_dec_36,tqwt_entropy_log_dec_28,tqwt_entropy_log_dec_35,app_LT_TKEO_std_2_coef,app_LT_entropy_shannon_3_coef,tqwt_minValue_dec_18,std_12th_delta,app_LT_TKEO_std_9_coef,tqwt_maxValue_dec_14,f2,...,app_LT_TKEO_mean_4_coef,DFA,mean_MFCC_4th_coef,app_entropy_log_7_coef,mean_MFCC_9th_coef,tqwt_entropy_log_dec_34,mean_MFCC_2nd_coef,det_LT_entropy_log_10_coef,app_LT_TKEO_mean_8_coef,class
1,18.9405,-3160.3491,-3190.1752,16.7003,-36938.1370,-0.61778,0.023057,4188.2456,0.105750,1031.849040,...,28.5949,0.71826,0.55527,231.9690,0.091821,-3726.2679,2.4874,-163.4186,793.0680,1.0
2,45.1780,-3212.9726,-3106.4317,16.5376,-36496.8101,-0.50757,0.032486,4148.9889,0.121580,1016.367294,...,28.3506,0.69481,0.45984,231.2326,0.236100,-3719.5566,2.8986,-166.2056,786.0763,1.0
3,4.7666,-3724.5248,-3082.5691,16.4817,-36288.7542,-0.47225,0.038433,4127.0967,0.081818,1032.406341,...,28.2014,0.67604,0.28918,230.7762,0.166150,-3661.5431,3.2208,-183.1261,782.0604,1.0
4,4.0603,-4965.0071,-1965.3808,16.8216,-35824.8476,-0.65225,0.037164,3910.7029,0.567740,1201.813897,...,26.2220,0.79672,-1.55900,226.5624,-0.153650,-2372.6055,3.1023,-37.7727,724.9443,1.0
5,6.1164,-5832.5871,-2102.3852,18.3595,-38399.3353,-0.62996,0.028731,4297.4639,0.567490,1215.346469,...,30.3224,0.79782,-1.75500,228.6784,-0.202080,-2484.7150,2.9451,-8.3446,827.9359,1.0


In [70]:
FPD.to_csv('FPD_95.csv', index=False)